# i210328 - Sarim Aeyzaz
# Assignment 2 and 3
# Timetable generation using Genetic Algorithm

In [62]:
import pandas as pd
import numpy as np
import random

df = pd.read_excel(r"C:\Users\hoxto\Desktop\courses.xlsx")
df

,S.#,Course,Section,Course Instructor
0,BS-2022,Object Oriented Programming (CS),CS-2B,Mr. Shehreyar Rashid
1,BS-2022,Object Oriented Programming (CS),CS-2C,Mr. Shehreyar Rashid
2,BS-2022,Object Oriented Programming (CS),CS-2G,Mr. Shehreyar Rashid
3,BS-2022,Object Oriented Programming (CS),CS-2D,Ms. Amna Irum
4,BS-2022,Object Oriented Programming (CS),CS-2E,Ms. Amna Irum
...,...,...,...,...
412,BS-2019,Digital Marketing (CS),CS-8A,Husbi Ahmed
413,BS-2019,Digital Marketing (CS),CS-8B,Husbi Ahmed
414,BS-2019,Technopreneurship (CS),CS-8A,Usman Ahmed
415,BS-2019,Technopreneurship (CS),CS-8B,Sadaf Zahra Usman


In [63]:
# Extract room names and timeslot names and create chromosomes based on course + section + instructor

timetable = pd.read_excel(r"C:\Users\hoxto\Desktop\timetable.xlsx")

rooms = timetable['Room'].values.tolist()[:30]
timeslots = timetable.columns.values.tolist()[1:]

chromosomes = np.array([[row['Course'], row['S.#']+'-'+row['Section'], row['Course Instructor']] for _, row in df.iterrows()])

print(rooms)
print(timeslots)
display(chromosomes)
print(chromosomes.shape)

['C-301', 'C-302', 'C-303', 'C-304', 'C-305', 'C-307', 'C-308', 'C-309', 'C-310', 'C-311', 'C-401', 'C-402', 'C-403', 'C-404', 'C-405', 'C-406', 'C-407', 'C-408', 'C-409', 'C-410', 'B-227', 'B-229', 'B-230', 'A-301', 'A-302', 'A-303', 'A-305', 'A-310', 'A-311', 'A-314']
['08:30-09:50', '10:00-11:20', '11:30-12:50', '01:00-02:20', '02:30-03:50', '03:55-05:15']


array([['Object Oriented Programming (CS)', 'BS-2022-CS-2B',
        'Mr. Shehreyar Rashid'],
       ['Object Oriented Programming (CS)', 'BS-2022-CS-2C',
        'Mr. Shehreyar Rashid'],
       ['Object Oriented Programming (CS)', 'BS-2022-CS-2G',
        'Mr. Shehreyar Rashid'],
       ...,
       ['Technopreneurship (CS)', 'BS-2019-CS-8A', 'Usman Ahmed'],
       ['Technopreneurship (CS)', 'BS-2019-CS-8B', 'Sadaf Zahra Usman'],
       ['Entrepreneurship (DS)', 'BS-2019-DS-N', 'Ayesha Yaqoob']],
      dtype='<U60')

(417, 3)


# Genetic Algorithm

In [64]:
populationSize = 30
generations = 100000
mutationProb = 0.2

population = [np.array([[random.choice(chromosomes) for _ in timeslots] for _ in rooms])  for _ in range(populationSize)]

In [65]:
population[0].T[1] # Column Sections
population[0].T[2] # Column Teachers

population[0][0].T[1] # Row sections
population[0][0].T[2] # Row teachers


array(['Ayesha Yaqoob', 'Eeman Mirza', 'Nagina Safdar',
       'Dr. Labiba Fahad', 'Dr. Abid Rauf', 'Dr. Isma ul Hassan'],
      dtype='<U60')

In [66]:
def fitness(timetable):
    
    score = 0

    # If a column doesn't have a section name clash, +40 points
    for x in timetable.T[1]:
        if len(np.unique(x)) == len(x):
            score += 40
            
    # If a column doesn't have a teacher name clash, +40 points    
    for x in timetable.T[2]:        
        if len(np.unique(x)) == len(x):
            score += 40
            
    # Collect sections and their counts
    arr = np.vstack(timetable[:]).T[2]
    sections, sectionCounts = np.unique(arr, return_counts = True)

    # Collect teachers and their counts
    arr2 = np.vstack(timetable[:]).T[2]
    teachers, teacherCounts = np.unique(arr2, return_counts = True)

    # 1 point per section having less than 4 occurunces
    # 1 point per teacher having less than 3 occurunces

    score += (sectionCounts < 5).sum()
    score += (teacherCounts < 4).sum()
                   
    return score

In [67]:
# Same as fitness scoring but with a boolean check than a score check (used to check if best generation fullfulls ALL
# contraints)

def checkFinal(timetable):
    
    check = True
    ls, ls2 = [], []
    
    for x in timetable.T[1]:
        if len(np.unique(x)) != len(x):
            ls.append("yea")
            check = False
    
    for x in timetable.T[2]:        
        if len(np.unique(x)) != len(x):
            ls2.append("yea")
            check = False
            
    arr = np.vstack(timetable[:]).T[2]
    sections, sectionCounts = np.unique(arr, return_counts = True)

    arr2 = np.vstack(timetable[:]).T[2]
    teachers, teacherCounts = np.unique(arr2, return_counts = True)

    # Debugging info
    print((sectionCounts > 4).sum(), (teacherCounts > 3).sum(), ls, ls2)

    if (sectionCounts > 4).sum() > 0:
        check = False
    if (teacherCounts > 3).sum() > 0:
        check = False
        
    return check

In [68]:
from numpy.random import choice

# Roulette wheel selection based on index (range of numbers b/w 0 and 50) and their respective scores
# scores are converted into a probability list summing to 1
def selection(index, scores):
    distribution = scores / scores.sum()
    return choice(index, 2, p = distribution, replace = False)

In [69]:
# Single point crossover
def crossover(parent1, parent2):
        
    # Swap the parents via rows by using a random divider line
    if random.random() < 0.5:
        index = random.randint(1,len(parent1)-2)

        child1 = np.concatenate((parent1[:index], parent2[index:]))
        child2 = np.concatenate((parent2[:index], parent1[index:]))

    # Swap the parents via columns by using a random divider line    
    else:
        index = random.randint(1,len(timeslots)-2)
        
        child1 = np.concatenate((parent1[:, :index], parent2[:, index:]), axis=1)
        child2 = np.concatenate((parent1[:, :index], parent2[:, index:]), axis=1)
        
    return [child1, child2]

In [70]:
def mutation(child, probability):
    
    # Only mutate if value is lower than set probability value
    if random.random() < probability:
        
        # Mutate either rows or column
        if random.random() < 0.5:
            # Mutate random rows of child
            for index in range(len(child)):
                if random.random() < probability:
                    child[index] = [random.choice(chromosomes) for _ in timeslots]
        else:

            # Mutate an entire column of child
            index = random.randint(0,len(timeslots)-1)
            child[:, index] = np.array([random.choice(chromosomes) for _ in rooms])
                
    return child

In [71]:
for gen in range(generations):
    
    # Calculate scores of current population
    scores = np.array([fitness(p) for p in population])
    newPopulation = []
    
    for i in range(populationSize // 2):
        
        # Select parents, children and apply mutation on children based on functions
        
        parents = selection(np.arange(0, populationSize), scores)
        children = crossover(population[parents[0]], population[parents[1]])
        
        children[0] = mutation(children[0], mutationProb)
        children[1] = mutation(children[1], mutationProb)

        # Append the children to the new population
        
        newPopulation.extend(children)
    
    # Combine newpopulation with old population and only keep the best 50 based on their score

    population = sorted(population + newPopulation, key=fitness, reverse=True)[:populationSize]

    # Output debugging data and stop when all constraints are fulfilled
    if gen % 100 == 0:
        print(f"Generation - {gen + 1}")
        print(f"Best Score: {fitness(population[0])}")
        print(scores)
        
        # Check to see is the best timetable fulfills all conditions given in the function
        # if true, then display it and break
        if checkFinal(population[0]):
            print("Timetable found meeting both hard and soft constraints!")
            break    

Generation - 1
Best Score: 295
[232 227 204 196 202 246 235 198 181 207 183 212 196 213 195 190 201 184
 250 178 260 197 237 186 200 207 205 184 204 281]
0 3 ['yea', 'yea', 'yea', 'yea', 'yea'] ['yea', 'yea', 'yea', 'yea', 'yea']
Generation - 101
Best Score: 521
[521 521 521 521 521 521 521 521 521 521 521 521 521 521 521 521 521 521
 521 521 521 521 521 521 521 521 521 521 521 521]
0 1 ['yea', 'yea', 'yea', 'yea', 'yea'] []
Generation - 201
Best Score: 521
[521 521 521 521 521 521 521 521 521 521 521 521 521 521 521 521 521 521
 521 521 521 521 521 521 521 521 521 521 521 521]
0 1 ['yea', 'yea', 'yea', 'yea', 'yea'] []
Generation - 301
Best Score: 533
[533 533 533 533 533 533 533 533 533 533 533 533 533 533 533 533 533 533
 533 533 533 533 533 533 533 533 533 533 533 533]
0 1 ['yea', 'yea', 'yea', 'yea'] ['yea']
Generation - 401
Best Score: 533
[533 533 533 533 533 533 533 533 533 533 533 533 533 533 533 533 533 533
 533 533 533 533 533 533 533 533 533 533 533 533]
0 1 ['yea', 'yea', 

Generation - 4601
Best Score: 687
[687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687
 687 687 687 687 687 687 687 687 687 687 687 687]
0 1 ['yea'] []
Generation - 4701
Best Score: 687
[687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687
 687 687 687 687 687 687 687 687 687 687 687 687]
0 1 ['yea'] []
Generation - 4801
Best Score: 687
[687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687
 687 687 687 687 687 687 687 687 687 687 687 687]
0 1 ['yea'] []
Generation - 4901
Best Score: 687
[687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687
 687 687 687 687 687 687 687 687 687 687 687 687]
0 1 ['yea'] []
Generation - 5001
Best Score: 687
[687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687
 687 687 687 687 687 687 687 687 687 687 687 687]
0 1 ['yea'] []
Generation - 5101
Best Score: 687
[687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687 687
 687 687 687 687 687 687 687 687 

In [72]:
# Display the timetable in a viewable format
finalTimeTable = pd.DataFrame([[" | ".join(cell) for cell in row] for row in population[0]], columns = timeslots, index = rooms)
finalTimeTable

,08:30-09:50,10:00-11:20,11:30-12:50,01:00-02:20,02:30-03:50,03:55-05:15
C-301,Networks and Cyber Security-II (CY) | BS-2020-...,Statistical Modelling | BS-2020-CS-6D | Dr. Sh...,Digital Marketing (DS) | BS-2020-DS-6N | Sidra...,Malware Analysis and Detection | MS Elective -...,Technical and Business Writing (All Programs) ...,Research Methodology | MS (CS)-MCS-A | Dr. Ham...
C-302,German Language (CS) | BS-2021-CS-4A | Afia Qazi,Theory of Automata | BS-2020 -CS-A | Ms. Tajwa...,DevOPs | BS -2019 -CS-8B | Mr. Saim Safdar,Applied Image Processing | PhD-PCS-3A | Dr. Ak...,Search-based Software Engineering | MS Electiv...,Networks and Cyber Security-II (CY) | BS-2020-...
C-303,Applied Programming | MS (CS)-MCS-A | M. Aadil...,Introduction to Software Engineering (SE) | BS...,Computer Networks (AI) | BS-2020-AI-6J | Dr. U...,Machine Learning (BS-AI) | BS-2020 -AI-A | Dr....,Introduction to Software Engineering (SE) | BS...,Islamic Studies/Ethics (DS) | BS-2022-DS-2B |...
C-304,Probability and Statistics (CS) | BS-2021-CS-4...,Database Systems (CS) | BS-2021-CS-4A/Y | Dr. ...,Communication and Presentation Skills (DS) | B...,Object Oriented Programming (CS) | BS-2022-CS-...,Islamic Studies/Ethics (SE) | BS-2022-SE-2G | ...,Communication and Presentation Skills (SE) | ...
C-305,Digital Logic Design (CS) | BS-2022-CS-2D | Mr...,Pakistan Studies (CS) | BS-2022-CS-2G | Gul-e-...,Object Oriented Programming (SE) | BS-2022-SE-...,Object Oriented Programming (CS) | BS-2022-CS-...,Differential Equations (CS) | BS-2022-CS-2B | ...,Probability and Statistics (CS) | BS-2021-CS-4...
C-307,Fundamentals of Software Engineering (CY) | BS...,Applied Evolutionary Computing (AI) | BS-2019-...,Programming Fundamentals (All Programs) | BS-2...,Digital Marketing (DS) | BS-2020-DS-6N | Sidra...,Communication and Presentation Skills (AI) | B...,Computer Networks (SE) | BS-2020-SE-6R | Mr. A...
C-308,Design and Analysis of Algorithms (CS) | BS-20...,Fundamentals of Management (CS) | BS-2021-CS-4...,Database Systems (CS) | BS-2021-CS-4C/D | Ms. ...,Programming Fundamentals (All Programs) | BS-2...,Applied Physics | BS-2022 -CS-A | Aisha Ijaz,Machine Learning (BS-AI) | BS-2020 -AI-A | Dr....
C-309,Model Based Testing | MS Elective -MCS-A | Dr....,Marketing Management (CS) | BS-2021-CS-4D | Ta...,Object Oriented Programming (CS) | BS-2022-CS-...,Database Systems (DS) | BS-2021-DS-4M | Dr. Wa...,Object Oriented Programming (CS) | BS-2022-CS-...,Pakistan Studies (CS) | BS-2022-CS-2C | Maimoo...
C-310,Communication and Presentation Skills (SE) | ...,Communication and Presentation Skills (CS) | B...,Object Oriented Programming (DS) | BS-2022-DS-...,Calculus and Analytical Geometry (All Programs...,Web Engineering (SE) | BS-2020-SE-6Q | Mr. Zah...,Communication and Presentation Skills (CS) | B...
C-311,Ethical Hacking Concepts and Practices | BS -2...,Communication and Presentation Skills (CS) | B...,Process Mining and Simulation (SE Supporting-I...,Software Engineering (CS) | BS-2020-CS-6A | Ms...,Discrete Structures | BS-2022-AI-2A | Ms. Sair...,German Language (SE) | BS-2021-SE-4Q | Ebrahim...


In [73]:
finalTimeTable.loc['C-301']

08:30-09:50    Networks and Cyber Security-II (CY) | BS-2020-...
10:00-11:20    Statistical Modelling | BS-2020-CS-6D | Dr. Sh...
11:30-12:50    Digital Marketing (DS) | BS-2020-DS-6N | Sidra...
01:00-02:20    Malware Analysis and Detection | MS Elective -...
02:30-03:50    Technical and Business Writing (All Programs) ...
03:55-05:15    Research Methodology | MS (CS)-MCS-A | Dr. Ham...
Name: C-301, dtype: object

In [74]:
finalTimeTable.loc['C-301', '08:30-09:50']

'Networks and Cyber Security-II (CY) | BS-2020-CY-6T | Dr. Qaiser Shafi'

In [75]:
finalTimeTable['08:30-09:50']

C-301    Networks and Cyber Security-II (CY) | BS-2020-...
C-302     German Language (CS) | BS-2021-CS-4A | Afia Qazi
C-303    Applied Programming | MS (CS)-MCS-A | M. Aadil...
C-304    Probability and Statistics (CS) | BS-2021-CS-4...
C-305    Digital Logic Design (CS) | BS-2022-CS-2D | Mr...
C-307    Fundamentals of Software Engineering (CY) | BS...
C-308    Design and Analysis of Algorithms (CS) | BS-20...
C-309    Model Based Testing | MS Elective -MCS-A | Dr....
C-310    Communication and Presentation Skills  (SE) | ...
C-311    Ethical Hacking Concepts and Practices | BS -2...
C-401    Communication and Presentation Skills (CS) | B...
C-402    Discrete Structures | BS-2022-AI-2A | Ms. Sair...
C-403    Data Structures (All Programs) | BS-2021 -4G |...
C-404    Data Structures (All Programs) | BS-2021 -4B |...
C-405    Probability and Statistics (CS) | BS-2021-CS-4...
C-406    Object Oriented Programming (CS) | BS-2022-CS-...
C-407    Software Engineering (CS) | BS-2020-CS-6F | Dr.